### Customer Analytics

Analyze a dataset containing customer information. Use EDA techniques to explore the data, identify trends, and create visualizations that show customer demographics, purchasing patterns, and correlations between variables.

In [ ]:
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Unzip the file
with zipfile.ZipFile('Customer Data.zip', 'r') as zip_ref:
    zip_ref.extractall('Customer_Data')

# Step 2: List all the files in the extracted folder
file_list = os.listdir('Customer_Data')
file_list

In [ ]:
# Load each CSV file into a DataFrame
portfolio = pd.read_csv('Customer_Data/portfolio.csv')
profile = pd.read_csv('Customer_Data/profile.csv')
transcript = pd.read_csv('Customer_Data/transcript.csv')

# List to hold the dataframes for easy reference
dfs = {'portfolio': portfolio, 'profile': profile, 'transcript': transcript}

# Iterate over each dataframe to explore the data
for name, df in dfs.items():
    print(f"\nExploring the {name} dataset:\n")
    
    # Check the basic information and data types of each column
    print("\nData Types and Non-Null Counts:\n")
    print(df.info())

    # Check for missing values
    print("\nMissing Values:\n")
    print(df.isnull().sum())

    # View basic statistics of numerical columns
    print("\nSummary Statistics for Numerical Columns:\n")
    print(df.describe())

    # Display unique values in categorical columns
    categorical_columns = df.select_dtypes(include='object').columns
    for col in categorical_columns:
        print(f"\nUnique values in '{col}':")
        print(df[col].unique())

    # Check for duplicate rows
    duplicates = df[df.duplicated()]
    print(f"\nNumber of duplicate rows in {name}: {len(duplicates)}")

    # Print a separator for readability
    print("\n" + "-" * 50)

In [ ]:
# Clean Data

In [ ]:
# Portfolio

# Drop 'Unnamed: 0' column
portfolio.drop(columns=['Unnamed: 0'], inplace=True)

# Convert 'channels' from string representation of list to actual list
import ast
portfolio['channels'] = portfolio['channels'].apply(ast.literal_eval)

# One-hot encode the 'channels' column
channels_dummies = portfolio['channels'].str.join('|').str.get_dummies()
portfolio = pd.concat([portfolio, channels_dummies], axis=1)

# Drop 'channels' after one-hot encoding
portfolio.drop(columns=['channels'], inplace=True)

# View cleaned portfolio data
print(portfolio.head())


In [ ]:
# Profile
# Drop 'Unnamed: 0' column
profile.drop(columns=['Unnamed: 0'], inplace=True)

# Convert 'became_member_on' to datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# Fill missing values for 'gender' with 'Unknown'
profile['gender'].fillna('Unknown', inplace=True)

# Fill missing values for 'income' with median
profile['income'].fillna(profile['income'].median(), inplace=True)

# View cleaned profile data
print(profile.head())


In [ ]:
# Transcript

# Drop 'Unnamed: 0' column
transcript.drop(columns=['Unnamed: 0'], inplace=True)

# Convert 'value' column from dictionary-like string to actual dictionary and expand into separate columns
transcript['value'] = transcript['value'].apply(ast.literal_eval)
value_df = pd.json_normalize(transcript['value'])
transcript = pd.concat([transcript.drop(columns=['value']), value_df], axis=1)

# View cleaned transcript data
print(transcript.head())

In [ ]:
# Visualization

# Gender Distribution
plt.figure(figsize=(8, 6))
sns.countplot(data=profile, x='gender', palette='viridis')
plt.title('Distribution of Gender Among Customers')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

# Age Distribution
plt.figure(figsize=(10, 6))
sns.histplot(data=profile, x='age', bins=30, kde=True, color='blue')
plt.title('Age Distribution of Customers')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

# Income Distribution
plt.figure(figsize=(10, 6))
sns.histplot(data=profile, x='income', bins=30, kde=True, color='green')
plt.title('Income Distribution of Customers')
plt.xlabel('Income')
plt.ylabel('Frequency')
plt.show()

# Step 2: Purchasing Patterns Analysis

# Transaction Analysis by Time
transcript_transactions = transcript[transcript['event'] == 'transaction']
transcript_transactions['time_days'] = transcript_transactions['time'] / 24  # Converting time to days
plt.figure(figsize=(12, 6))
sns.histplot(data=transcript_transactions, x='time_days', bins=30, kde=True, color='orange')
plt.title('Distribution of Transactions Over Time')
plt.xlabel('Time (in days)')
plt.ylabel('Number of Transactions')
plt.show()

# Offer Viewed vs Offer Completed
offer_events = transcript[transcript['event'].isin(['offer viewed', 'offer completed'])]
plt.figure(figsize=(10, 6))
sns.countplot(data=offer_events, x='event', palette='coolwarm')
plt.title('Comparison of Offers Viewed and Completed')
plt.xlabel('Event')
plt.ylabel('Count')
plt.show()

# Step 3: Correlations Between Variables (Revised Heatmap)

# Select only numeric columns for correlation analysis
profile_numeric = profile.select_dtypes(include=['int64', 'float64'])

# Correlation Heatmap (for Profile Dataset)
plt.figure(figsize=(10, 8))
sns.heatmap(profile_numeric.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Customer Profile Data (Numeric Columns)')
plt.show()